### 1 - Import test data

In [1]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-3-12", end="2024-4-12", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-12 00:00:00,1.093374,1.093494,1.093255,1.093374,1.093374,0
2024-03-12 00:15:00,1.093374,1.093494,1.093374,1.093494,1.093494,0
2024-03-12 00:30:00,1.093494,1.093494,1.093255,1.093374,1.093374,0
2024-03-12 00:45:00,1.093374,1.093374,1.093135,1.093255,1.093255,0
2024-03-12 01:00:00,1.093135,1.093255,1.093016,1.093016,1.093016,0
...,...,...,...,...,...,...
2024-04-11 22:45:00,1.073076,1.073076,1.072961,1.072961,1.072961,0
2024-04-11 23:00:00,1.072961,1.073192,1.072961,1.073192,1.073192,0
2024-04-11 23:15:00,1.073192,1.073192,1.073076,1.073076,1.073076,0


### 2 - Define your signal function

In [2]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [3]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    2035
2      74
1      67
Name: count, dtype: int64

### 3 - Connect to the market and execute trades

In [4]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [5]:
from config import access_token, accountID
def get_candles(n):
    #access_token='XXXXXXX'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [6]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [7]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

      Open    Close     High      Low
0  1.06709  1.06592  1.06712  1.06559
1  1.06592  1.06556  1.06637  1.06553
1.0672599999999999    1.06474    1.0639    1.06642
{'orderCreateTransaction': {'id': '4', 'accountID': '101-001-28870641-001', 'userID': 28870641, 'batchID': '4', 'requestID': '61227683848778711', 'time': '2024-04-12T09:51:36.077376463Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.06726', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.06474', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '5', 'accountID': '101-001-28870641-001', 'userID': 28870641, 'batchID': '4', 'requestID': '61227683848778711', 'time': '2024-04-12T09:51:36.077376463Z', 'type': 'ORDER_FILL', 'orderID': '4', 'instrument': 'EUR_USD', 'units': '1000', 'requestedUnits': '1000', 'price': '1.06581', 'pl': '0.0000', 'quotePL': '0', 'finan